In [ ]:
#Import everything needed for matplotlib
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
#Import of libraries to be used
import numpy as np
import pandas as pd
import datetime as dt

In [ ]:
 # Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
#Create the engine
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
#Create the base and automap the classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

In [ ]:
#Now to assign the classes to 
Station = Base.classes.station
Measurement = Base.classes.measurement

In [ ]:
#Create a session
session = Session(engine)

In [ ]:
first_row_station = session.query(Station).first()
first_row_station.__dict__

In [ ]:
first_row_measurement = session.query(Measurement).first()
first_row_measurement.__dict__

In [ ]:
#The date is presented in a text format which is another wrinkle to work with
#First we can use strftime and some functions to find the most recent date
top_date = session.query(func.max(func.strftime("%Y-%m-%d", Measurement.date)))

#From this we can pull out out a a string of the date
top_date_date = dt.datetime.strptime(top_date[0][0], "%Y-%m-%d")

#Now I need a variable that is the date 1 year ago. I used weeks=52.2 since 52*7 /= 365
year_ago = top_date_date - dt.timedelta(weeks=52.2)

In [ ]:
#Now To put my query into a variable
qry = session.query(Measurement.date, Measurement.prcp).\
                        filter(Measurement.date >= year_ago)

#Making a dataframe from the query
measurement_df = pd.read_sql(qry.statement, session.bind)

#Set the date to the index
measurement2_df = measurement_df.set_index('date')

#Finally to sort
measurement3_df = measurement2_df.sort_index(ascending = True)

measurement3_df.tail()

In [ ]:
#Now to make a pretty little graph
fig, ax = plt.subplots(figsize = (10, 7))
measurement3_df.plot(ax = ax, x_compat = True,rot=90)
ax.set_ylabel('Inches')

In [ ]:
#Print out the statistics for the precipitation
measurement3_df.describe()

In [ ]:
first_row_station = session.query(Station).first()
first_row_station.__dict__

In [ ]:
#Query to count the total number of stations
qry_station_count = session.query(Station.station).group_by(Station.station).count()
qry_station_count
                      

In [ ]:
#Make a query to find the activity of each station reporting
qry_station_active = session.query(Measurement.station, func.count(Measurement.station)).\
                        group_by(Measurement.station).\
                        order_by(func.count(Measurement.station).desc()).all()

#Here is my list, there are many like it, but this one is mine
print(qry_station_active)

#The first entry is the most active since this is descending order
most_active = qry_station_active[0][0]
print("\n" + "The most active station is " + str(most_active))

In [ ]:
temperature_readings = session.query(func.min(Measurement.tobs), func.max(Measurement.tobs), func.avg(Measurement.tobs)).\
                                     filter(Measurement.station == most_active).all()
temperature_readings

In [ ]:
qry_temp = session.query(Measurement.date, Measurement.tobs).\
                        filter(Measurement.date >= year_ago).\
                        filter(Measurement.station == most_active)

#Making a dataframe from the query
temperature_df = pd.read_sql(qry_temp.statement, session.bind)


In [ ]:
#Now to make a pretty little graph
fig, ax = plt.subplots(figsize = (10, 7))
temperature_df.plot.hist(ax = ax, bins = 12)
ax.set_ylabel('Frequency')
ax.set_xlabel('Temperature')